In [1]:
import sys
sys.path.append('..')

from models.compression.vq_vae import VQVAE
from dataset import ImageCompressionDataModule

/home/vedpatel/.conda/envs/pyml/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = VQVAE(
    h_dim=56,
    res_h_dim=128,
    n_res_layers=5,
    n_embeddings=256 * 4,
    embedding_dim=1,
    beta=0.25,
    lr=1e-3,
)

data_module = ImageCompressionDataModule(
    data_dir="../../data/PetImages",
)
data_module.setup()

Setting up [LPIPS] perceptual loss: trunk [squeeze], v[0.1], spatial [off]
Loading model from: /home/vedpatel/.conda/envs/pyml/lib/python3.11/site-packages/lpips/weights/v0.1/squeeze.pth


/home/vedpatel/.conda/envs/pyml/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/vedpatel/.conda/envs/pyml/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SqueezeNet1_1_Weights.IMAGENET1K_V1`. You can also use `weights=SqueezeNet1_1_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
import torch
from tqdm import tqdm

for batch in tqdm(data_module.train_dataloader()):
    x, _ = batch

    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    model = model.to(device)
    x = x.to(device)
    model.eval()

    with torch.no_grad():
        _, x_recon, _ = model(x)

    x_vis = (x + 1) / 2
    x_recon_vis = (x_recon + 1) / 2

    loss = 1 - model.criterion(x_recon_vis, x_vis)
    
    if loss.isnan().any():
        print("NaN detected in loss")

  2%|▏         | 116/5616 [00:13<10:42,  8.56it/s]


KeyboardInterrupt: 